In [2]:
import numpy as np
import pandas as pd

In [16]:
col_names = {'long_trip', 'lead_time', 'avg_price_per_room', 'special_requests_no', 'month', 'prev_show_rate', 'canceled'}
df = pd.read_csv("C:/Users/aakas/Desktop/Team-A/hotel_reservations_finalclean.txt")
df = df.drop('adult_no', axis = 1).drop('child_no', axis = 1)
df.head(10)

,weekend_nights_no,weekday_nights_no,lead_time,avg_price_per_room,special_requests_no,month,prev_show_rate,canceled
0,1,2,224,65.00,0,10.0,0.5,0
1,2,3,5,106.68,1,11.0,0.5,0
2,2,1,1,60.00,0,2.0,0.5,1
3,0,2,211,100.00,0,5.0,0.5,1
4,1,1,48,94.50,0,4.0,0.5,1
5,0,2,346,115.00,1,9.0,0.5,1
6,1,3,34,107.55,1,10.0,0.5,0
7,1,3,83,105.61,1,12.0,0.5,0
8,0,4,121,96.90,1,7.0,0.5,0
9,0,5,44,133.44,3,10.0,0.5,0


In [18]:
class Node():
    def __init__(self, feature_index=None, threshold=None, left=None, right=None, info_gain=None, value=None):
        self.feature_index = feature_index
        self.threshold = threshold
        self.left = left
        self.right = right
        self.info_gain = info_gain

        self.value = value

In [19]:
class DecisionTreeClassifier():
    def __init__(self, min_samples_split=2, max_depth=2):
        ''' constructor '''

        self.root = None
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth

    def build_tree(self, dataset, curr_depth=0):
        X, Y = dataset[:,:-1], dataset[:,-1]
        num_samples, num_features = np.shape(X)

        if num_samples>=self.min_samples_split and curr_depth<=self.max_depth:
            best_split = self.get_best_split(dataset, num_samples, num_features)
            if best_split["info_gain"]>0:
                left_subtree = self.build_tree(best_split["dataset_left"], curr_depth+1)
                right_subtree = self.build_tree(best_split["dataset_right"], curr_depth+1)

                return Node(best_split["feature_index"], best_split["threshold"],
                            left_subtree, right_subtree, best_split["info_gain"])
            
        leaf_value = self.calculate_leaf_value(Y)

        return Node(value=leaf_value)
    
    def get_best_split(self, dataset, num_samples, num_features):
        best_split = {}
        max_info_gain = -float("inf")

        for feature_index in range(num_features):
            feature_values = dataset[:, feature_index]
            possible_thresholds = np.unique(feature_values)

            for threshold in possible_thresholds:

                dataset_left, dataset_right = self.split(dataset, feature_index, threshold)
                if len(dataset_left)>0 and len(dataset_right)>0:
                    y, left_y, right_y = dataset[:, -1], dataset_left[:, -1], dataset_right[:, -1]
                    curr_info_gain = self.information_gain(y, left_y, right_y, "gini")
                    if curr_info_gain>max_info_gain :
                        best_split["feature_index"] = feature_index
                        best_split["threshold"] = threshold
                        best_split["dataset_left"] = dataset_left
                        best_split["dataset_right"] = dataset_right
                        best_split["info_gain"] = curr_info_gain
                        max_info_gain = curr_info_gain

        return best_split
    
    def split(self, dataset, feature_index, threshold):
        dataset_left = np.array([row for row in dataset if row[feature_index]<=threshold])
        dataset_right = np.array([row for row in dataset if row[feature_index]>threshold])
        return dataset_left, dataset_right
    
    def information_gain(self, parent, l_child, r_child, mode="entropy"):

        weight_l = len(l_child) / len(parent)
        weight_r = len(r_child) / len(parent)
        if mode == "gini":
            gain = self.gini_index(parent) - (weight_l*self.gini_index(l_child) + weight_r*self.gini_index(r_child))
        else:
            gain = self.entropy(parent) - (weight_l*self.entropy(l_child) + weight_r*self.entropy(r_child))
        return gain
    
    def entropy(self, y):
        class_labels = np.unique(y)
        entropy = 0
        for cls in class_labels:
            p_cls = len(y[y == cls]) / len(y)
            entropy += -p_cls * np.log2(p_cls)
        return entropy
    
    def gini_index(self, y):

        class_labels = np.unique(y)
        gini = 0
        for cls in class_labels:
            p_cls = len(y[y == cls]) / len(y)
            gini += p_cls**22
        return 1 - gini
    
    def calculate_leaf_value(self, Y):

        Y = list(Y)
        return max(Y, key=Y.count)
    
    def print_tree(self, tree=None, indent=" "):
        if not tree:
            tree=self.root
        
        if tree.value is not None:
            print(tree.value)

        else:
            print("X_"+str(tree.feature_index), "<=", tree.threshold, "?", tree.info_gain)
            print("%sleft:" % (indent), end="")
            self.print_tree(tree.left, indent + indent)
            print("%sright:" % (indent), end="")
            self.print_tree(tree.right, indent + indent)
        
    def fit(self, X, Y):
        dataset = np.concatenate((X, Y), axis=1)
        self.root = self.build_tree(dataset)

    def predict(self, X):
        predictions = [self.make_prediction(x, self.root) for x in X]
        return predictions
    def make_prediction(self, x, tree):
        if tree.value!=None: return tree.value
        feature_val = x[tree.feature_index]
        if feature_val <= tree.threshold:
            return self.make_prediction(x, tree.left)
        else:
            return self.make_prediction(x, tree.right)

        

Train-Test Split

In [20]:
X = df.iloc[:, :-1].values
Y = df.iloc[:, -1].values.reshape(-1, 1)
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=.2, random_state=41)

Fit the Model

In [21]:
classifier = DecisionTreeClassifier(min_samples_split=3, max_depth=3)
classifier.fit(X_train, Y_train)
classifier.print_tree()

X_2 <= 8.0 ? 0.021855436809389173
 left:X_4 <= 0.0 ? 0.09072690691726781
  left:X_5 <= 11.0 ? 0.07984464927012913
    left:X_6 <= 0.5 ? 0.07612087277774082
        left:0.0
        right:0.0
    right:0.0
  right:X_4 <= 1.0 ? 0.06840091292447126
    left:X_3 <= 76.0 ? 0.07460737321457589
        left:0.0
        right:0.0
    right:X_1 <= 4.0 ? 0.03259032397578796
        left:0.0
        right:0.0
 right:X_4 <= 2.0 ? 0.020873788610599964
  left:X_5 <= 1.0 ? 0.010437281853561853
    left:X_2 <= 28.0 ? 0.12453428401955075
        left:0.0
        right:0.0
    right:X_6 <= 0.86 ? 0.005038894862810173
        left:0.0
        right:0.0
  right:0.0


In [22]:
Y_pred = classifier.predict(X_test)
from sklearn.metrics import accuracy_score
accuracy_score(Y_test, Y_pred)

0.670709855272226